In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer
import torch
from torch import nn
from transformers import BertModel
import numpy as np
from torch.optim import Adam
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os

df = pd.read_csv("/kaggle/input/feedback-prize-2021/train.csv")
df

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
labels = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6
          }
prev_label = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6,
          'Start':7
          }

In [3]:
sequence = df['predictionstring']
sequence = sequence.apply(lambda x: x.split())
sequence = sequence.apply(lambda x: list(map(int, x)))
sequence
df_new = df
start, end = [],[]
for i in range(len(sequence)):
    start.append(sequence[i][0])
    end.append(sequence[i][-1])
df_new['Start'] = start
df_new['End'] = end

prev = [7]

for i in range(1, len(sequence)):
    if df_new['End'][i-1] < df_new['Start'][i]:
        prev.append(prev_label[df['discourse_type'][i-1]])
    else:
        prev.append(7)

In [4]:
df_new['one_hot'] = prev
df_new

In [5]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['discourse_type']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 256, truncation=True,
                                return_tensors="pt") for text in df['discourse_text']]
        self.prev = [one_hot for one_hot in df['one_hot']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def get_batch_prev(self, idx):
        return self.prev[idx]
    
    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_prev = self.get_batch_prev(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_prev, batch_y

In [19]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 7)
        self.linear_2 = nn.Linear(15,7)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask, prev_identity):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        linear_output = torch.cat((linear_output, prev_identity), 1)
        #print(linear_output)
        linear_output = self.linear_2(linear_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

print(len(df[:70000]))


In [8]:
np.random.seed(112)
df_new_new = df[:70000]
df_train, df_val, df_test = np.split(df_new_new.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_new_new)), int(.9*len(df_new_new))])

print(len(df_train),len(df_val), len(df_test))

In [16]:
model

In [20]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, prev ,train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                train_label=train_label.to(torch.int64)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                prev = torch.nn.functional.one_hot(prev, num_classes=8).to(device)

                output = model(input_id, mask, prev)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, prev_val ,val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    prev_val = torch.nn.functional.one_hot(prev_val, num_classes=8).to(device)

                    output = model(input_id, mask, prev)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 4
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

In [21]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, prev, test_label in test_dataloader:
            
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)
            prev = torch.nn.functional.one_hot(prev, num_classes=8).to(device)
            
            output = model(input_id, mask, prev)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)